In [8]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path
import pyarrow

import importlib  
fa_etl = importlib.import_module("fa-etl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
county = "36061"

path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
# taxhist = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
# valhist = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/ValHist{county}.parquet")
# annual = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
# sales = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")
merged = pl.read_parquet(path_to_dir+f"dev/{county}/unified/merged.parquet")

In [10]:
merged.shape

(75519, 22)

In [11]:
merged.head(10)

PropertyID,Year,Value,AssessmentUsed,MarketTotalValue,ApprTotalValue,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,SaleAmt,TaxAmt,TaxAmtAdjusted
i64,i64,i64,str,i64,i64,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,f64
91847870,2018,117677,"""Assd""",261504,null,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",1580000,1310424,13104.24
91849008,2018,247162,"""Assd""",1462000,null,"""R""",2024-02-07,40.750447,-73.997723,"""362 W 30TH ST""","""NEW YORK""","""NY""","""10001""","""36061""","""009700""","""4000""","""B""","""36061_9700""",4400000,3114052,31140.52
91848046,2018,71155,"""Assd""",158123,null,"""R""",2024-02-23,40.738643,-73.987954,"""254 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",712775,773352,7733.52
91846549,2018,149310,"""Assd""",331800,null,"""R""",2024-02-07,40.729262,-74.004785,"""63 DOWNING ST …","""NEW YORK""","""NY""","""10014""","""36061""","""006700""","""2002""","""B""","""36061_6700""",2475000,1527372,15273.72
91847537,2018,128574,"""Assd""",285721,null,"""R""",2024-02-07,40.73889,-73.987904,"""260 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",1625000,1496148,14961.48
91848333,2018,356895,"""Assd""",793100,null,"""R""",2024-02-07,40.726433,-73.99315,"""40 BOND ST APT…","""NEW YORK""","""NY""","""10012""","""36061""","""005502""","""1002""","""B""","""36061_5502""",7400000,3958776,39587.76
91846649,2018,219589,"""Assd""",487975,null,"""R""",2024-02-07,40.7408,-73.9989,"""22-26 W 15TH S…","""NEW YORK""","""NY""","""10011""","""36061""","""008100""","""2000""","""5""","""36061_8100""",3100000,2518932,25189.32
91847215,2018,299300,"""Assd""",665110,null,"""R""",2024-02-23,40.7408,-73.9989,"""12-14 W 18TH S…","""NEW YORK""","""NY""","""10011""","""36061""","""008100""","""2000""","""5""","""36061_8100""",2825000,3327036,33270.36
91846134,2022,718067,"""Assd""",1595704,null,"""R""",2024-02-07,40.740314,-73.988185,"""23 E 22ND ST #…","""NEW YORK""","""NY""","""10010""","""36061""","""005600""","""2003""","""B""","""36061_5600""",12500000,8808528,88085.28


In [12]:
merged.filter([
    pl.col("SaleAmt").is_not_null(),
    pl.col("Year") > 2017
]).select(
    ['PropertyID','Year','Value','AssessmentUsed','SaleAmt', 'TaxAmt', 'TaxAmtAdjusted']
).with_columns([
    (pl.col('TaxAmt')/pl.col('Value')).alias("ApproxTaxRate")
])

PropertyID,Year,Value,AssessmentUsed,SaleAmt,TaxAmt,TaxAmtAdjusted,ApproxTaxRate
i64,i64,i64,str,i64,i64,f64,f64
91847870,2018,117677,"""Assd""",1580000,1310424,13104.24,11.13577
91849008,2018,247162,"""Assd""",4400000,3114052,31140.52,12.599235
91848046,2018,71155,"""Assd""",712775,773352,7733.52,10.868555
91846549,2018,149310,"""Assd""",2475000,1527372,15273.72,10.229536
91847537,2018,128574,"""Assd""",1625000,1496148,14961.48,11.636474
91848333,2018,356895,"""Assd""",7400000,3958776,39587.76,11.092271
91846649,2018,219589,"""Assd""",3100000,2518932,25189.32,11.471121
91847215,2018,299300,"""Assd""",2825000,3327036,33270.36,11.116057
91846134,2022,718067,"""Assd""",12500000,8808528,88085.28,12.267
